# Práctica: Transformación y Enriquecimiento de Datos

## 1. Carga de Datos (Simulación E-commerce)
Trabajaremos con un DataFrame que simula un reporte de ventas diarias. Contiene precios, costos, categorías y códigos abreviados que necesitan ser traducidos.

In [ ]:
import pandas as pd
import numpy as np

# Datos simulados de ventas
data = {
    'producto': ['Laptop Gamer', 'Mouse Inalámbrico', 'Monitor 4K', 'Teclado Mecánico', 'USB 32GB', 'Silla Ergonómica'],
    'precio_venta': [1200.00, 25.50, 300.00, 85.00, 10.00, 250.00],
    'costo_unitario': [800.00, 10.00, 180.00, 40.00, 2.00, 150.00],
    'categoria': ['Electronics', 'Accessories', 'Electronics', 'Accessories', 'Storage', 'Furniture'],
    'codigo_pago': ['CC', 'PP', 'CC', 'DB', 'CS', 'PP'], # CC: Credit Card, PP: PayPal, etc.
    'cliente_vip': [True, False, True, False, False, True]
}

df = pd.DataFrame(data)

print("--- Dataset de Ventas Inicial ---")
display(df)

## 2. Creación de Nuevas Columnas (Operaciones Vectorizadas)
Pandas nos permite operar columnas enteras como si fueran números simples.
**Ejemplo:** Vamos a calcular la **Ganancia (Profit)** y el **Margen de ganancia** por producto.

In [ ]:
# 1. Crear columna de Ganancia Neta (Precio - Costo)
df['ganancia_neta'] = df['precio_venta'] - df['costo_unitario']

# 2. Crear columna de Margen (%)
# Fórmula: (Ganancia / Precio Venta) * 100
df['margen_porcentaje'] = (df['ganancia_neta'] / df['precio_venta']) * 100

# Redondeamos a 2 decimales para que se vea limpio
df['margen_porcentaje'] = df['margen_porcentaje'].round(2)

print("--- Datos con nuevas métricas financieras ---")
display(df[['producto', 'precio_venta', 'ganancia_neta', 'margen_porcentaje']])

## 3. Transformaciones complejas con `apply()` y `lambda`
A veces la lógica no es una simple suma o resta. Usamos `apply` con una función `lambda` cuando queremos aplicar una condición "fila por fila".

**Caso de uso:**
1. Aplicar un **Impuesto (IVA)**: 16% para Electrónicos, 0% para el resto.
2. Etiquetar el producto como "High End" si cuesta más de $100.

In [ ]:
# EJEMPLO 1: Lógica condicional compleja
# Si la categoría es 'Electronics', el impuesto es 16%, si no, es 0.
df['impuesto_calculado'] = df.apply(
    lambda row: row['precio_venta'] * 0.16 if row['categoria'] == 'Electronics' else 0,
    axis=1
)

# EJEMPLO 2: Etiquetado de texto basado en números
# "Premium" si precio > 100, sino "Económico"
df['tipo_producto'] = df['precio_venta'].apply(lambda x: "Premium" if x > 100 else "Económico")

print("--- Resultado de la lógica condicional ---")
display(df[['producto', 'categoria', 'precio_venta', 'impuesto_calculado', 'tipo_producto']])

## 4. Reemplazo de Valores (`map` vs `replace`)
Ideal para limpiar datos categóricos o traducir códigos.
* **`map()`**: Usa un diccionario para transformar **toda** una columna. Lo que no esté en el diccionario se convierte en `NaN`.
* **`replace()`**: Busca y reemplaza valores específicos, respetando los que no encuentre.

In [ ]:
# Diccionario de traducción de códigos de pago
diccionario_pagos = {
    'CC': 'Tarjeta de Crédito',
    'PP': 'PayPal',
    'DB': 'Tarjeta de Débito',
    'CS': 'Efectivo'
}

# Usamos MAP para traducir la columna completa
df['metodo_pago_completo'] = df['codigo_pago'].map(diccionario_pagos)

# Usamos REPLACE para cambiar un valor específico en la columna 'categoria'
# Cambiamos 'Storage' por 'Almacenamiento'
df['categoria'] = df['categoria'].replace('Storage', 'Almacenamiento')

print("--- Datos traducidos y estandarizados ---")
display(df[['producto', 'codigo_pago', 'metodo_pago_completo', 'categoria']])

## 📌 Validación de Transformaciones
Verificamos que los cálculos y transformaciones tengan sentido lógico.

In [ ]:
# 1. Validar que la ganancia no sea negativa (en este caso hipotético)
assert (df['ganancia_neta'] >= 0).all(), "Error: Hay ganancias negativas."

# 2. Validar que el mapeo funcionó (no deben quedar nulos en metodo_pago_completo)
assert df['metodo_pago_completo'].isnull().sum() == 0, "Error: Falló el mapeo de pagos."

# 3. Validar lógica de apply (Si es Premium, precio debe ser > 100)
check_premium = df[df['tipo_producto'] == 'Premium']
assert (check_premium['precio_venta'] > 100).all(), "Error: La etiqueta Premium está mal asignada."

print("✅ Todas las transformaciones pasaron las pruebas lógicas.")
display(df.head())